# <span style="color:green; font-family:Georgia; font-size:2.5m;"> Deep learning Pytorch - Vote Americain

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# <span style="color:green; font-family:Georgia; font-size:1.5m;"> 1- Préparation des données

# <span style="color:green; font-family:Georgia; font-size:0.8em;"> 1. Etiquetage

In [2]:
fichier = open("house-votes-84.txt", "r")
lignes = fichier.readlines()

# republican = 1 ,democrat = -1 ,y = 1, n = -1 et ? = 0
def choixVote(ch):
    if ch == 'republican':
        return 1
    elif ch == 'democrat':
        return 0
    elif ch == 'y':
        return 1
    elif ch == 'n':
        return -1
    else:
        return 0

In [3]:
def parti(n):
    if n == 1:
        return 'republican'
    else:
        return 'democrat'

In [4]:
Vote = []
for ligne in lignes:
    ligne = ligne.replace('\n','')
    t = ligne.split(',')
    for i in range(0,len(t)):
        Vote.append(choixVote(t[i]))

Vote = np.reshape(Vote,(-1,17))
np.random.seed(987654321)
np.random.shuffle(Vote)
# print(Vote)

In [5]:
Target = Vote[:,0]
Data = Vote[:,1:]

In [6]:
# print(Vote.shape)
# print(Target.shape)
# print(Data.shape)

# <span style="color:green; font-family:Georgia; font-size:0.8em;"> 2. Separation des données

In [7]:
nbExempleTrain = 335
TrainData = Data[0:nbExempleTrain,:]
TrainTarget = Target[0:nbExempleTrain]
# print(TrainData)
# print(TrainTarget)

In [8]:
TestData = Data[nbExempleTrain:,:]
TestTarget = Target[nbExempleTrain:]
# print(TestData)
# print(TestTarget)

In [9]:
import torch
import torch.utils.data as utils
from torchvision import datasets, transforms
import torch.nn.functional as F

In [10]:
torch.set_default_tensor_type(torch.FloatTensor)
# print(torch.get_default_dtype())
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])


train_data_tensor = torch.tensor(TrainData)
train_target_tensor = torch.tensor(TrainTarget) # transform to torch tensors

test_data_tensor = torch.tensor(TestData) # transform to torch tensors
test_target_tensor = torch.tensor(TestTarget) # transform to torch tensors

train_dataset = utils.TensorDataset(train_data_tensor,train_target_tensor) # create your datset
test_dataset = utils.TensorDataset(test_data_tensor,test_target_tensor) # create your datset

batch_size = 1
# Set the training loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Set the testing loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [11]:
# print(train_data_tensor)
# print(train_target_tensor)
# print(test_data_tensor)
# print(test_target_tensor)

In [12]:
weights = torch.randn(16, 2, requires_grad=True)

In [13]:
# print(weights)

In [14]:
def test(weights, test_loader):
    test_size = len(test_loader.dataset)
    correct = 0

    for batch_idx, (data, target) in enumerate(test_loader):
        #print(batch_idx, data.shape, target.shape)
#         data = data.view((-1, 28*28))
        #print(batch_idx, data.shape, target.shape)
        data = data.float()
        target = target.to(dtype=torch.long)
        outputs = torch.matmul(data, weights)
        softmax = F.softmax(outputs, dim=1)
        pred = softmax.argmax(dim=1, keepdim=True)
        n_correct = pred.eq(target.view_as(pred)).sum().item()
        correct += n_correct

    acc = correct / test_size
    print(" Accuracy on test set", acc)
    return

test(weights, test_loader)

 Accuracy on test set 0.51


In [15]:
it = 0
for batch_idx, (data, targets) in enumerate(train_loader):
    # Be sure to start the loop with zeros grad
    if weights.grad is not None:
        weights.grad.zero_()
    
    #print("batch_idx: {}, data.shape: {}, target.shape: {}".format(batch_idx, data.shape, targets.shape))
    data = data.view((-1, 16))
    data = data.float()
    targets = targets.to(dtype=torch.long)
    outputs = torch.matmul(data, weights)
    #print("outputs.shape: {}".format(outputs.shape))

    log_softmax = F.log_softmax(outputs, dim=1)
    #print("Log softmax: {}".format(log_softmax.shape))

    #print((-log_softmax[0][targets[0]] + -log_softmax[1][targets[1]] )  / 2 )
    #print(-log_softmax[0][targets[0]], targets[0])
    
    loss = F.nll_loss(log_softmax, targets)
    print("\rLoss shape: {}".format(loss), end="")
    
    # Compute the gradients for each variables
    loss.backward()
    
    with torch.no_grad():
        weights -= 0.1*weights.grad
        
    it += 1
#     if it % 10 == 0:
    test(weights, test_loader)
        
    if it > 5000:
        break

Loss shape: 1.9687613248825073 Accuracy on test set 0.69
Loss shape: 1.866485834121704 Accuracy on test set 0.82
Loss shape: 0.03798047453165054 Accuracy on test set 0.82
Loss shape: 0.08352985978126526 Accuracy on test set 0.83
Loss shape: 0.04495957866311073 Accuracy on test set 0.82
Loss shape: 0.011565777473151684 Accuracy on test set 0.82
Loss shape: 0.017488909885287285 Accuracy on test set 0.82
Loss shape: 0.01873956248164177 Accuracy on test set 0.82
Loss shape: 0.12534119188785553 Accuracy on test set 0.81
Loss shape: 0.005033677443861961 Accuracy on test set 0.81
Loss shape: 0.012632353231310844 Accuracy on test set 0.81
Loss shape: 0.000813034363090992 Accuracy on test set 0.81
Loss shape: 0.07847686856985092 Accuracy on test set 0.81
Loss shape: 0.4597819149494171 Accuracy on test set 0.82
Loss shape: 0.0009777533123269677 Accuracy on test set 0.82
Loss shape: 0.12110688537359238 Accuracy on test set 0.83
Loss shape: 0.002839107532054186 Accuracy on test set 0.83
Loss shape

Loss shape: 0.01456826739013195 Accuracy on test set 0.94
Loss shape: 0.007539507932960987 Accuracy on test set 0.94
Loss shape: 0.000631848000921309 Accuracy on test set 0.94
Loss shape: 0.001320800743997097 Accuracy on test set 0.94
Loss shape: 0.2671017348766327 Accuracy on test set 0.93
Loss shape: 0.0005905076395720243 Accuracy on test set 0.93
Loss shape: 0.01204750221222639 Accuracy on test set 0.93
Loss shape: 0.04049219936132431 Accuracy on test set 0.93
Loss shape: 0.0002978596603497863 Accuracy on test set 0.93
Loss shape: 0.032047610729932785 Accuracy on test set 0.93
Loss shape: 0.00011157367407577112 Accuracy on test set 0.93
Loss shape: 0.002119439421221614 Accuracy on test set 0.93
Loss shape: 0.010463143698871136 Accuracy on test set 0.93
Loss shape: 0.003308420069515705 Accuracy on test set 0.93
Loss shape: 0.00023982033599168062 Accuracy on test set 0.93
Loss shape: 0.005127375479787588 Accuracy on test set 0.93
Loss shape: 0.6931471824645996 Accuracy on test set 0.9

Loss shape: 0.014392142184078693 Accuracy on test set 0.94
Loss shape: 0.0046680523082613945 Accuracy on test set 0.94
Loss shape: 0.684684693813324 Accuracy on test set 0.94
Loss shape: 0.0059931338764727116 Accuracy on test set 0.94
Loss shape: 0.005512394476681948 Accuracy on test set 0.94
Loss shape: 0.003874890971928835 Accuracy on test set 0.94
Loss shape: 0.0010081215295940638 Accuracy on test set 0.94
Loss shape: 0.0021158705931156874 Accuracy on test set 0.94
Loss shape: 0.038081809878349304 Accuracy on test set 0.94
Loss shape: 0.08827655762434006 Accuracy on test set 0.94
Loss shape: 0.19737423956394196 Accuracy on test set 0.94
Loss shape: 1.4781842764932662e-05 Accuracy on test set 0.94
Loss shape: 0.002864070236682892 Accuracy on test set 0.94
Loss shape: 1.042283535003662 Accuracy on test set 0.94
Loss shape: 0.3091065585613251 Accuracy on test set 0.95
Loss shape: 0.03758399188518524 Accuracy on test set 0.95
Loss shape: 0.5492416620254517 Accuracy on test set 0.96
Loss

In [21]:
batch_idx, (data, target) = next(enumerate(test_loader))
data = data.float()
print(data)
print(target)
outputs = torch.matmul(data, weights)
softmax = F.softmax(outputs, dim=1)
pred = softmax.argmax(dim=1, keepdim=True)

print("valeur réelle  : ", parti(target.numpy()[0]))
print("valeur predite : ", parti(pred[0].numpy()[0]))

tensor([[-1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,
         -1.,  1.]])
tensor([0], dtype=torch.int32)
valeur réelle  :  democrat
valeur predite :  republican
